In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import cm
import operator
import matplotlib.ticker as ticker
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

sns.set_context('poster', font_scale = 1.5)
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque'
savepath = os.path.join(path, 'figures')

In [ ]:
dat = pd.read_csv(os.path.join(path, 'unionizes', 'plaque_densities_7-13.csv'))

In [ ]:
dat.head()

In [ ]:
dat_count = pd.read_csv(os.path.join(path, 'plaque counts', 'plaque_counts_7-13.csv'))

In [ ]:
dat_count = dat_count.merge(dat[['image_series_id', 'structure_acronym', 'structure_volume']], 
                on=['image_series_id', 'structure_acronym'])

In [ ]:
dat_count['count_per_mm_3'] = dat_count['sum_plaques']/dat_count['structure_volume']

In [ ]:
dat_count.head()

In [ ]:
meta = pd.read_csv(os.path.join(path, 'filtered_plaque_dataset_07_20_2018.csv'))

In [ ]:
dataset = meta[meta['Plaque map call (R hemisphere)'] == 'Pass']
print(len(dataset))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
summary_structures = st.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]
ss_acronyms = [item['acronym'] for item in summary_structures]
ont_order = [item['graph_order'] for item in summary_structures]
ont_order_dict = dict(zip(ss_acronyms, ont_order))
print(len(summary_structure_ids))


In [ ]:
coarse_structures = st.get_structures_by_set_id([687527670])
coarse_structure_ids = [item['id'] for item in coarse_structures]
coarse_structure_acronyms = [item['acronym'] for item in coarse_structures]
ft = st.get_structures_by_acronym(['fiber tracts'])
coarse_structure_ids.append(ft[0]['id'])
coarse_structure_acronyms.append(ft[0]['acronym'])
ont_order = [item['graph_order'] for item in coarse_structures]
ont_order.append(ft[0]['graph_order'])
oo_dict_coarse = dict(zip(coarse_structure_acronyms, ont_order))
print(len(coarse_structure_ids))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.iteritems()}

In [ ]:
structures = summary_structure_ids + coarse_structure_ids + [997]

In [ ]:
dataset_ids = dat['image_series_id'].unique()
print(len(dataset_ids))

In [ ]:
isids = [int(value) for value in dataset['Link to image series']]

In [ ]:
dataset['image_series_id'] = isids

In [ ]:
dataset = dataset[dataset['image_series_id'].isin(dataset_ids)]
print(len(dataset))

In [ ]:
dataset.groupby(['Mouse Line', 'age group']).count()

In [ ]:
dat['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat['structure_acronym']]
dat_count['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat_count['structure_acronym']]

In [ ]:
def get_structure_data(age, structure, q_type):
    if q_type == 'density':
        data = dat
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = dat_count
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})    
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

def get_structure_data_tau(age, structure, q_type):
    if q_type == 'density':
        data = dat
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = dat_count
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})    
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in tau_dataset[tau_dataset['age group'] == age]['Mouse Line'].unique():
        isids = tau_dataset[(tau_dataset['Mouse Line'] == group) & 
                            (tau_dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

In [ ]:
tau_dataset = dataset[dataset['Mouse Line'].isin(['APP+/Tau+', 'APP+/Tau-'])]
dataset = dataset[~dataset['Mouse Line'].isin(['APP+/Tau+', 'APP+/Tau-'])]

In [ ]:
mouse_line_colors = ['#8da0cb', '#fc8d62', '#66c2a5']

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']

ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        if group != 'APP/PS1/rTg4510':
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
                isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})
density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]

ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        if group != 'APP/PS1/rTg4510':
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                             (dat_count['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
                isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})

fig, ax = plt.subplots(1, 2, figsize = (18, 7))
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat, ax=ax[0], s=10,
                 order = order, palette = mouse_line_colors, hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'])
#g.set_ylim([0, 0.016])
g.legend_.remove()
g.set_ylabel('Plaque Percent Volume')
g.set_title('Brain-Wide Plaque Density')
h = sns.swarmplot('age', 'plaque count', hue = 'group', 
                  data=count_dat, ax=ax[1], s=10, order = order,
                 palette = mouse_line_colors, hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'])
h.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
g.legend(loc=2, borderaxespad=0.)
h.legend(loc=2, borderaxespad=0.)
h.set_title('Brain-Wide Plaque Count')
fig.subplots_adjust(wspace = 0.3)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
def find_density_by_structure(structure):
    structure_id = ia_map[structure]
    ages = []
    groups = []
    plaque_density = []
    isids_carrythrough = []
    for age in dataset['age group'].unique():
        for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
                isids_carrythrough.append(isid)
    density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                                 'group': groups, 'plaque density': plaque_density,
                                 'image_series_id': isids_carrythrough})
    density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]
    return(density_dat)

def find_count_by_structure(structure):
    structure_id = ia_map[structure]
    ages = []
    groups = []
    plaque_count = []
    isids_carrythrough = []
    for age in dataset['age group'].unique():
        for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                             (dat_count['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
                isids_carrythrough.append(isid)
    count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                                 'group': groups, 'plaque count': plaque_count,
                                 'image_series_id': isids_carrythrough})
    return(count_dat)

In [ ]:
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']
root_d = find_density_by_structure('root')
root_c = find_count_by_structure('root')

iso_d = find_density_by_structure('Isocortex')
iso_c = find_count_by_structure('Isocortex')

hpc_d = find_density_by_structure('HPF')
hpc_c = find_count_by_structure('HPF')

fig, ax = plt.subplots(3, 2, sharex = True, figsize = (18, 20))
a = sns.swarmplot('age', 'plaque density', hue = 'group', data=root_d, ax=ax[0, 0], s=7,
                 order = order, palette = mouse_line_colors)
a.set_ylabel('Plaque % Volume')
a.legend(loc=2, borderaxespad=0.)
a.set_title('Brain-Wide Plaque Density')
b = sns.swarmplot('age', 'plaque count', hue = 'group', data=root_c, ax=ax[0, 1], s=7,
                  order = order, palette = mouse_line_colors)
b.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
b.set_title('Brain-Wide Plaque Count')
b.legend_.remove()

c = sns.swarmplot('age', 'plaque density', hue = 'group', data=iso_d, ax=ax[1, 0], s=7,
                 order = order, palette = mouse_line_colors)
c.set_ylabel('Plaque % Volume')
c.legend_.remove()
c.set_title('Plaque Density in Isocortex')
d = sns.swarmplot('age', 'plaque count', hue = 'group', data=iso_c, ax=ax[1, 1], s=7,
                  order = order, palette = mouse_line_colors)
d.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
d.legend_.remove()
d.set_title('Plaque Count in Isocortex')

e = sns.swarmplot('age', 'plaque density', hue = 'group', data=hpc_d, ax=ax[2, 0], s=7,
                 order = order, palette = mouse_line_colors)
e.set_ylabel('Plaque % Volume')
e.legend_.remove()
e.set_title('Plaque Density in Hippocampus')
f = sns.swarmplot('age', 'plaque count', hue = 'group', data=hpc_c, ax=ax[2, 1], s=7,
                  order = order, palette = mouse_line_colors)
f.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
f.legend_.remove()
f.set_title('Plaque Count in Hippocampus')

plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide iso hipp 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
density_dat = find_density_by_structure('root')
g = sns.factorplot('age', 'plaque density', hue = 'group', 
                   data=density_dat[density_dat['group'] != 'APP/PS1/rTg4510'], s=7, 
                   size = 5, aspect = 4, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors, legend = False)
g.set(yscale="log")
#ax.set_ylim([0.001, 10])
g.set_yticklabels(['0.001', '0.01', '0.1', '1', '10'])
plt.title('Whole Brain Plaque Density')
plt.legend(loc='upper left')
ax = g.ax
ax.set_ylabel('Percent Plaque Coverage')
ax.set_xlabel('')
plt.savefig(os.path.join(savepath, 'whole brain plaque density by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
ccf_colors = ['#70ff71', '#9ad2bd', '#7ed04b', '#8ada87', '#98d6f9', '#8599cc', '#ff7080', '#e64438', '#ff64ff', 
              '#ff9b88','#ff9bcd', '#f0f080', '#cccccc']

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque density', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.7])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.6))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2.5))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Percent Plaque Coverage')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 12 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque density', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.7])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.6))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2.5))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Percent Plaque Coverage')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 18 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
iso = structure_tree.descendant_ids([iso['id']])[0]
iso = [structure for structure in iso if structure in summary_structure_ids]

In [ ]:
plot_order = ['FRP', 'MOp', 'MOs', 'SSp-n', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-ul', 'SSp-tr', 'SSp-un',
              'SSs', 'GU', 'VISC', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpl',
              'VISpm', 'VISli', 'VISpor', 'ACAd', 'ACAv', 'PL', 'ILA', 'ORBl', 'ORBm', 'ORBvl', 'AId', 'AIp',
              'AIv', 'RSPagl', 'RSPd', 'RSPv', 'VISa', 'VISrl', 'TEa', 'PERI', 'ECT']

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   order = plot_order, palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
plt.xticks(rotation = 90)
plt.title('Plaque Volume in Isocortex Structures (18 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   order = plot_order, palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
plt.legend(loc=1, bbox_to_anchor = (0.95, 1))
ax.legend_.remove()
#ax.set_ylim([0, 3])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
#ax2.set_ylim([0, 1.5])
ax.legend(loc=2)
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 18 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   order = plot_order, palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (12 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   order = plot_order, palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
ax.legend_.remove()
#ax.set_ylim([0, 2])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
ax2.set_ylim([0, 0.25])
ax.legend(loc=2, bbox_to_anchor = (0, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 12 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   order = plot_order, palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('12 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration
mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   order = plot_order, palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  order = plot_order, hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (12 months)')
ax.legend_.remove()
#ax.set_ylim([0, 2])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 12 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('6 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   order = plot_order, palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('6 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration
mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   order = plot_order, palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  order = plot_order, hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (6 months)')
ax.legend_.remove()
ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 6 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
hipp = st.get_structures_by_acronym(['HPF'])[0]
hipp_ids = st.descendant_ids([hipp['id']])[0]
hipp_ss = [ids for ids in hipp_ids if ids in summary_structure_ids]
hipp_ss.remove(ia_map['FC'])
hipp_ss.remove(ia_map['IG'])
print(len(hipp_ss))

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 5))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
#plt.xticks(rotation = 90)
plt.title('Plaque Volume in Hippocampal Structures (18 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
plt.legend(loc=1, bbox_to_anchor = (0.95, 1))
ax.legend_.remove()
#ax.set_ylim([0, 3])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
#ax2.set_ylim([0, 1.5])
ax.legend(loc=2, bbox_to_anchor = (0.01, 1))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampal structures 18 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 5))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
#plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Hippocampus (12 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
ax.legend_.remove()
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
ax2.set_ylim([0, 0.3])
ax.legend(loc = 2, bbox_to_anchor = (0.02, 1))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampal structures 12 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('6 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('6 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration

mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.title('Percent Plaque Coverage in Hippocampus (6 months)')
ax.legend_.remove()
#ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus structures 6 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('12 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration

mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   palette = ['#52607f'], ax = ax)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.title('Percent Plaque Coverage in Hippocampus (12 months)')
ax.legend_.remove()
#ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2)
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus structures 12 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
def get_mean_value_per_structure(group, age, structure_ids):
    means = []
    isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
    for structure_id in structure_ids:
        str_mean = np.mean(dat[(dat['structure_id'] == structure_id) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        means.append(str_mean*100)
    structuredat = dict(zip(structure_ids, means))
    return structuredat, len(isids)

In [ ]:
all_structure_vals, n = get_mean_value_per_structure('Tg2576', '12 mo', dat.structure_id.unique())

In [ ]:
reference_space =  mcc.get_reference_space()
structure_vals, n = get_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('APP/PS1', '18 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 plaque map 18 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '18 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2.5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 18 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 10
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '18 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 18 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0])
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2])), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1]))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i])
    plt.axis('off')
cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
plt.savefig(os.path.join(savepath, 'ARA with plaque map.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)